# Analysing steps in a short walk using acceleration and rotation

In this noteboook we examine a recording from the Arduino IMU of a short walk with the aim to extract the step count, cadence and timings including ground time and step duration.

We implement three techniques outlined in the paper `A comprehensive comparison of simple step counting techniques using wrist- and ankle-mounted accelerometer and gyroscope signals` by Matthew Rudy and Joseph Mahoney - [https://www.researchgate.net/publication/325451208_A_comprehensive_comparison_of_simple_step_counting_techniques_using_wrist-_and_ankle-mounted_accelerometer_and_gyroscope_signals](https://www.researchgate.net/publication/325451208_A_comprehensive_comparison_of_simple_step_counting_techniques_using_wrist-_and_ankle-mounted_accelerometer_and_gyroscope_signals).

* Peak-finding
* Fast Fourier Transform (FFT)
* Autocorrelation

Each of these methods allows us to count steps. The peak-finding method also identifies where the steps occur in the timeseries, so this in turn allows us to isolate steps and calculate such things as ground time and step duration.

## The IMU

The Arduino Nano inertial measurement unit gives us acceleration, measured in `g`s (`1g = 9.8m/s/s`), and rotation (angular velocity). In this notebook we demonstrate how to extract steps from either acceleration or rotation.

## The data

We expect a CSV file with columns for time, 3 axes of acceleration, and 3 axes of gyroscopic rotation.

Example:
```
time,aX,aY,aZ,gX,gY,gZ
363328,1.0157471,-0.0056152344,0.032714844,1.0986328,-4.8828125,-3.540039
363338,1.0213623,0.009643555,0.017700195,1.8310547,-3.540039,-4.0893555
363348,1.015625,0.01940918,0.020874023,1.8920898,-2.1362305,-4.3945312
```

## Setup

In [ ]:
import pandas as pd
import seaborn as sns
from scipy.integrate import cumtrapz
from scipy.signal import butter, filtfilt, periodogram, spectrogram, find_peaks
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(11, 4)})
import numpy as np
import gpxpy
from xml.etree import ElementTree as ET
from datetime import timedelta
from tqdm.notebook import tqdm
tqdm()

## Data import

In [ ]:
data_file_path = "../data/20200914 mellor sports field/mellor sports field circles (wmf).csv"
output_file_path = "../data/20200914 mellor sports field/mellor sports field circles (wmf).gpx"
df = pd.read_csv(data_file_path)
df.columns = ["time", "aX", "aY", "aZ", "gX", "gY", "gZ"]
df.time = df.time - df.time.min()
df

## Key parameters

`fs` is a sampling frequency of an IMU used (i.e. how many times per second accelerationa and angular velocity are measured). The default delay between readings (defined in the Arduino code) is 10 ms, which corresponds to the sampling frequency of 100 Hz. The value of sampling frequency needs to be changed accordingly if different rate was used during the recording (either with a different device, or with modified Arduino code).

In [ ]:
fs = 1000 / 10 # Hz, sampling frequency
total_time = df.time.max() - df.time.min()

## Helper functions

In [ ]:
def get_time_period(a, b):
    """
    a, b -- time in seconds
    """
    return df.loc[(df.time >= a * 1000) & (df.time < b * 1000)]

def filter_series(data, fc_low=None, fc_high=None, fs=fs):
    if fc_low is not None and fc_high is not None:
        return band_pass(data, fc_low, fc_high, fs)
    elif fc_low is not None:
        return high_pass(data, fc_low, fs)
    elif fc_high is not None:
        return low_pass(data, fc_high, fs)
    else:
        return data
        
def band_pass(data, fc_low, fc_high, fs):
    w_low = fc_low / (fs / 2) # Normalize the frequency
    w_high = fc_high / (fs / 2) # Normalize the frequency
    b, a = butter(5, [w_low, w_high], 'bandpass')
    return filtfilt(b, a, data)

def high_pass(data, fc_low, fs):
    w_low = fc_low / (fs / 2) # Normalize the frequency
    b, a = butter(5, w_low, 'highpass')
    return filtfilt(b, a, data)

def low_pass(data, fc_high, fs):
    w_high = fc_high / (fs / 2) # Normalize the frequency
    b, a = butter(5, w_high, 'lowpass')
    return filtfilt(b, a, data)

def get_magnitude(data, fc_low=None, fc_high=None, fs=None):
    magnitude = np.sqrt((data**2).sum(axis=1))
    if fc_low is not None and fc_high is not None:
        return band_pass(magnitude, fc_low, fc_high, fs)
    elif fc_low is not None:
        return high_pass(magnitude, fc_low, fs)
    elif fc_high is not None:
        return low_pass(magnitude, fc_high, fs)
    else:
        return magnitude
    
def peak_detection_steps(data, pos_kwargs=None, neg_kwargs=None, plot=False):
    peaks, _ = find_peaks(data, **pos_kwargs)
    neg_peaks, _ = find_peaks(-data, **neg_kwargs)
    if plot:
        sns.lineplot(x=range(len(data)), y=data)
        sns.scatterplot(x=peaks, y=data[peaks])
        sns.scatterplot(x=neg_peaks, y=data[neg_peaks])
        plt.show()
#     return len(peaks)
    return max(len(peaks), len(neg_peaks))

def fft_dominant_freq(data, fs, plot=False):
    f, Pxx = periodogram(data, fs=fs)
    if plot:
        sns.lineplot(f, Pxx)
        plt.xlim([0.0,10.0])
        plt.show()
    return f[np.argmax(Pxx)]

def fft_steps(data, dt, fs, plot=False):
    return dt * fft_dominant_freq(data, fs, plot) / 1000

def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[:int(len(result)/2)]

def autocorr_steps(data, plot=False):
    corr = autocorr(data)
    peaks, _ = find_peaks(corr)
    if plot:
        sns.lineplot(x=range(len(corr)), y=corr)
        sns.scatterplot(x=peaks, y=corr[peaks])
        plt.show()
    return len(peaks)

def to_steps_per_minute(step_count, dt):
    """
    dt -- time in seconds
    """
    return step_count / dt * 60

def get_spm_for_period(a, b, columns, fc_low=None, fc_high=None, peak_detection_kwargs={}):
    frame = get_time_period(a, b)
    dt = (frame.time.max() - frame.time.min()) / 1000.0
    aMagnitude = get_magnitude(frame.loc[:,columns], fc_low=fc_low, fc_high=fc_high, fs=fs)
    n_peak_steps = peak_detection_steps(
        aMagnitude, 
        **peak_detection_kwargs
    )
    dominant_freq = fft_dominant_freq(aMagnitude, fs=fs)
    n_autocorr_steps = autocorr_steps(aMagnitude)
    return pd.Series({
        "peak_detection_spm": to_steps_per_minute(n_peak_steps, dt), 
        "fft_spm": dominant_freq * 60, 
        "autocorrelation_spm": to_steps_per_minute(n_autocorr_steps, dt), 
    })

def get_spm(data, a, b, peak_detection_fc=(None, None), fft_fc=(None, None), autocorrelation_fc=(None, None), peak_detection_kwargs={}):
    frame = get_time_period(a, b)
    dt = (frame.time.max() - frame.time.min()) / 1000.0
    
    _data = filter_series(data.loc[frame.index].values, *peak_detection_fc)
        
    n_peak_steps = peak_detection_steps(
        _data, 
        **peak_detection_kwargs
    )
    
    _data = filter_series(data.loc[frame.index].values, *fft_fc)
    dominant_freq = fft_dominant_freq(_data, fs=fs)
    
    _data = filter_series(data.loc[frame.index].values, *autocorrelation_fc)
    n_autocorr_steps = autocorr_steps(_data)
    return pd.Series({
        "peak_detection_spm": to_steps_per_minute(n_peak_steps, dt), 
        "fft_spm": dominant_freq * 60, 
        "autocorrelation_spm": to_steps_per_minute(n_autocorr_steps, dt), 
    })

def scalar_projection(u, v):
    """
    project u on v
    """
    v_norm = np.sqrt(sum(v**2))
    return (np.dot(u, v)/v_norm)



# Measuring steps from acceleration

## Explore
We first look at what data we have. 

In [ ]:
sns.lineplot(x=df.time, y=df.aX, label="aX")
sns.lineplot(x=df.time, y=df.aY, label="aY")
sns.lineplot(x=df.time, y=df.aZ, label="aZ")

Zooming in, we can clearly see how different components of acceleration contribute to a step

In [ ]:
_df = df.iloc[4050:4090].loc[:, ["aX", "aY", "aZ"]]
_df.plot()

## Preparation
For further calculations, we need to find and remove the gravity vector. We can do this by taking a mean average of the first readings (assuming the user was initially standing still)

In [ ]:
_df = df.iloc[:9]
sns.lineplot(x=_df.time, y=_df.aX, label="aX")
sns.lineplot(x=_df.time, y=_df.aY, label="aY")
sns.lineplot(x=_df.time, y=_df.aZ, label="aZ")

In [ ]:
gravity_vector = _df.loc[:,["aX", "aY", "aZ"]].mean()
(gravity_vector, np.sqrt((gravity_vector**2).sum()))

The resulting vector is of the expected direction (primarily in Z+ direction) and the expected magnitude (roughly 1 `g`). Subtracting it from all the acceleration readings leaves only the unbalanced acceleration/force (i.e. the acceleration that contributes to motion). We also multiply acceleration values by 9.8 to convert it to m/s^2

In [ ]:
df.loc[:,["aX", "aY", "aZ"]] = (df.loc[:,["aX", "aY", "aZ"]] - gravity_vector) * 9.8

Since we do not know the direction of movement (the device can be placed in different orientation), we perform Principal component analysis (PCA). The 0th component should ideally lie in the direction of movement, assuming there's a greater variation in acceleration in this direction.

In [ ]:
df.loc[:,["pca0", "pca1", "pca2"]] = PCA().fit_transform(df.loc[:,["aX", "aY", "aZ"]])

As we expected, the 0th component shows two clearly defined peaks that correspond to a step.

In [ ]:
_df = df.iloc[4050:4090].loc[:, ["pca0", "pca1", "pca2"]]
_df.plot()

We plot the spectrogram of the 0th component, which shows a signal at ~1.3 Hz (roughly 78 steps per minute). However the second harmonic of this signal (at ~2.6 Hz or 156 steps per minute) is more prominent, which is the expected value for cadence measured from both feet.

In [ ]:
f, t, Sxx = spectrogram(df.pca0, fs)
plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.ylim([0,10])
plt.show()

## Cadence calculations
We use three methods to estimate cadence from 0th component:

* Peak detection -- count all peaks (positive and negative) with minimum height of 35 m/s^2 and distance of 20 readings between (~200 ms)
* FFT (Fast Fourier Transform) -- perform FFT (i.e. transform signal from time domain to a frequency domain) and take a frequency with the highest power
* Autocorrelation -- perform a correlation of a signal with a delayed copy of itself (this reveals periodicity of a signal) and count the peaks.

Additionally, we filter the signal differently for different methods. We apply bandpass filter with frequencies 0.5-4.0 Hz for FFT, and 0.3-4.0 Hz for Autocorrelation. No filtering is done before Peak detection, as it makes the peaks that we've seen before less prominent and makes it harder to detect them.

Applying the methods to a short segment of 5 seconds gives 85.7, 192.0 and 160.7 steps per minute respectively. Note: since the second harmonic is more prominent, FFT and autocorrelation give cadence for both feet (i.e. double of peak detection method)

In [ ]:
a = 4500
time = df.time[a:a+500]
total_time = time.max() - time.min()
display(to_steps_per_minute(peak_detection_steps(
    df.pca0[a:a+500].values, 
    pos_kwargs={
        "distance": 20,
        "height": (35, None)
    }, 
    neg_kwargs={
        "distance": 20,
        "height": (35, None)
    },
    plot=True
), total_time / 1000))
display(to_steps_per_minute(fft_steps(band_pass(df.pca0[a:a+500], fc_low=0.5, fc_high=4.0, fs=fs), dt=total_time, fs=fs, plot=True), total_time / 1000))
display(to_steps_per_minute(autocorr_steps(band_pass(df.pca0[a:a+500], fc_low=0.3, fc_high=4.0, fs=fs), plot=True), total_time / 1000))

Applying these algorithms to windows of 10 seconds, we get a series of cadence measurements over time. The expected mean cadence (measured with Stryd) is ~82 spm. The results of our calculations come close to that with 71.5, 170.2 and 154.5 spm for Peak detection, FFT, and Autocorrelation respectively (as before FFT and Autocorrelation gives cadence for both feet, hence double).

The cadence line plot looks choppy, since the number of steps measured is a discrete. However it can be smoothen using Exponential Moving Average (see lines labeled with EMA suffix).

In [ ]:
peak_detection_kwargs = {
    "pos_kwargs": {
        "prominence": 20,
        "distance": 20,
        "height": (35, None)
    }, 
    "neg_kwargs": {
        "prominence": 20,
        "distance": 20,
        "height": (35, None)
    },
}

spms = []
dt = 10
step = 1.0
for a in tqdm(np.arange(0, df.time.max() / 1000, step)):
    b = a + dt
    spm = get_spm(df.pca0, a, b, fft_fc=(0.5, 4.0), autocorrelation_fc=(0.3, 4.0), peak_detection_kwargs=peak_detection_kwargs)
    spm.name = a
    spms += [spm]
#     break
acceleration_spm_df = pd.DataFrame(spms)
acceleration_spm_df["peak_detection_spm_EMA"] = acceleration_spm_df.peak_detection_spm.ewm(alpha=0.3, adjust=False).mean()
acceleration_spm_df["autocorrelation_spm_EMA"] = acceleration_spm_df.autocorrelation_spm.ewm(alpha=0.3, adjust=False).mean()
acceleration_spm_df.plot()
display(acceleration_spm_df.describe())
plt.legend()
plt.show()

# Measuring steps from rotation
The same approach can be taken to calculate cadence from gyroscope data (angular velocity)

In [ ]:
sns.lineplot(x=df.time, y=df.gX, label="gX")
sns.lineplot(x=df.time, y=df.gY, label="gY")
sns.lineplot(x=df.time, y=df.gZ, label="gZ")

In [ ]:
_df = df.iloc[4050:4090].loc[:, ["gX", "gY", "gZ"]]
_df.plot()

Instead of principal components, we are using the total magnitude of angular velocity

In [ ]:
_magnitude = get_magnitude(df.loc[:,["gX", "gY", "gZ"]].iloc[4050:4090], fs=fs)
sns.lineplot(x=df.time.iloc[4050:4090], y=_magnitude)

Even though the signal is weaker for angular velocity magnitude, the peaks at 1.3 and 2.6 Hz are still detectable

In [ ]:
f, t, Sxx = spectrogram(get_magnitude(df.loc[:,["gX", "gY", "gZ"]], fs=fs), fs)
plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.ylim([0,10])
plt.show()

And hence, applying the cadence measurement algorithms as before, we get the mean cadence of 65.9, 89.3, and 75.8 spm for Peak detection, FFT and Autocorrelation respectively. Which are close to the expected ~82 spm measured with Stryd.

In [ ]:
peak_detection_kwargs = {
    "pos_kwargs": {
        "height": (50, None)
    }, 
    "neg_kwargs": {
        "height": (50, None)
    },
}

spms = []
dt = 10
step = 1
for a in np.arange(0, df.time.max() / 1000, step):
    b = a + dt
    spm = get_spm_for_period(a, b, ["gX", "gY", "gZ"], fc_low=0.5, fc_high=2.0, peak_detection_kwargs=peak_detection_kwargs)
    spm.name = a
    spms += [spm]
spm_df = pd.DataFrame(spms)
spm_df.plot()
spm_df.describe()


# Export GPX
The resulting can be exported as a GPX track, to use it other tools that support GPX (e.g. DC Rainmaker Analyzer). By default Peak detection cadence calculated from acceleration and smoothen with EMA is exported. 

Note: the timestamp of the first reading needs to be provided, since GPX assumes that all timestamps are absolute, but the Wearable My Foot device only gives relative timestamps

In [ ]:
def get_cadence_extension(cadence):
    prefix = "gpxtrx:"
    element = ET.Element(f"{prefix}TrackPointExtension")
    cadence_element = ET.SubElement(element, f"{prefix}cad")
    # Schema only permits integers up to 254
    cadence_element.text = str(int(cadence if cadence <= 254 else 254))
    return element

def get_point(time, cadence):
    extensions = [get_cadence_extension(cadence)]
    point = gpxpy.gpx.GPXTrackPoint()
    point.extensions = extensions
    point.time = time
    return point

def get_gpx(timestamps, cadence):
    """
    data -- pandas DataFrame with time and cadence fields
    """
    gpx = gpxpy.gpx.GPX()
    gpx.nsmap["gpxtrx"] = 'http://www.garmin.com/xmlschemas/GpxExtensions/v3'
    track = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(track)
    segment = gpxpy.gpx.GPXTrackSegment()
    track.segments.append(segment)
    segment.points = [get_point(t, c) for t, c in zip(timestamps, cadence)]
    return gpx
    
    
timestamp = pd.to_datetime("2020-09-14T17:07:14")
timestamps = [timestamp + timedelta(seconds=x) for x in acceleration_spm_df.index]
cadence = acceleration_spm_df.peak_detection_spm_EMA

with open(output_file_path, 'w+') as f:
    f.write(get_gpx(timestamps, cadence).to_xml())